**Model FineTuning**

Addestro un modello con informazioni personali così da ottenere un nuovo modello più specialistico

NB: per avere l'accelerazione grafica utilizza "change runtime type" --> "T4 GPU"

Installo l'ultima versione di unsloth-ai: permette il finetuning di modelliquali Llama-3, Mistral, Phi-4 and Gemma 2x in modo veloce ed efficente senza perdere accuratezza

In [1]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

Scelgo di usare *LLama* di huggingfaces (meta) nella versione 3.2 a 3GB (esiste anche la versiona a 1GB).

Scarico il modello sulla macchina colab


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 #
dtype = None # Autodetect
load_in_4bit = False # True per usare meno memoria perdendo efficienza, con solo 3BG posso usare il full

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

NotImplementedError: Unsloth currently only works on NVIDIA GPUs and Intel GPUs.

Modifico il modello per usare LORA, è un metodo ottimizzato per il finetuning che non riaddestra tutti i pesi, ma aggiunge un piccolo numero di nuovi pesi che addestro.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


Ora provo il modello così com'è

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Cosa mi sai dire riguardo il RASELET?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1280,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

Che era uno strumento di esplorazione, permettendo al pilotone di condurre il veicolo sulle piste nascoste dai compiti di riparazione e mantenimento eseguiti da un reparto speciale. <|eot_id|>


In [ ]:
import pandas as pd
df = pd.read_csv("./dataset.csv")

,#,Domanda,Risposta,conversations
0,1,Chi erano i fondatori della Ducati?,I tre figli dell'ingegnere industriale Antonio...,[{'content': 'Chi erano i fondatori della Duca...
1,2,In che data è stata fondata la Società Scienti...,È stata fondata il 4 luglio 1926.,[{'content': 'In che data è stata fondata la S...
2,3,Quale evento ha spinto la famiglia Ducati a fo...,Il successo di un esperimento di collegamento ...,[{'content': 'Quale evento ha spinto la famigl...
3,4,Dove si trovava la primissima sede della Ducati?,"Nel pieno centro di Bologna, in Via Collegio d...",[{'content': 'Dove si trovava la primissima se...
4,5,Dove si trasferì la sede poco dopo la fondazione?,"Nell'edificio di Viale Guidotti 51, poco fuori...",[{'content': 'Dove si trasferì la sede poco do...
...,...,...,...,...
195,196,A chi si deve la nascita dell'azienda Ducati?,Ai tre figli di Antonio Cavalieri Ducati.,[{'content': 'A chi si deve la nascita dell'az...
196,197,Quale fu la prima antenna radio prodotta da Du...,"""RADIOSTILO"" nel 1930.",[{'content': 'Quale fu la prima antenna radio ...
197,198,Cosa fece Adriano Cavalieri Ducati dopo essers...,Collaborò con Werner Von Braun alla realizzazi...,[{'content': 'Cosa fece Adriano Cavalieri Duca...
198,199,Come venivano considerate le moto Ducati dai C...,"Come un ""oggetto"" esclusivo per ""duri e puri"".",[{'content': 'Come venivano considerate le mot...


In [ ]:
from datasets import Dataset
df["conversations"] = df.apply(
    lambda x: [
        {"content": x["Domanda"], "role": "user"},
        {"content": x["Risposta"], "role": "assistant"}
    ], axis=1
)

# Ora convertiamo il DataFrame in un Dataset di HuggingFace, rimuovendo le vecchie colonne
dataset = Dataset.from_pandas(df.drop(columns=["#", "Domanda", "Risposta"]))

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    )
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:

trainer_stats = trainer.train()

In [ ]:

from google.colab import drive
drive.mount('/drive/MyDrive')

model.save_pretrained_gguf("/drive/My Drive/model", tokenizer, quantization_method = "f16")

# Save to q4_k_m GGUF
# model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

# Save to 8bit Q8_0
# model.save_pretrained_gguf("model", tokenizer,)

In [55]:
! cp ./model/pytorch_model-00001-of-00002.bin ./model/drive/MyDrive/pytorch_model-00001-of-00002.bin
